In [357]:
import pandas as pd
import numpy as np

In [358]:
recent_matches = pd.read_csv('recent_seasons_unnormalized.csv')
labels = pd.read_csv('labels_recent_seasons.csv')

recent_matches = recent_matches.replace(-1, np.nan)
labels = labels.replace(-1, np.nan)

In [359]:
home_odds_labels = ['B365H', 'BWH', 'IWH', 'LBH', 'PSH', 'WHH', 'SJH', 'VCH', 'GBH', 'BSH']
draw_odds_labels = ['B365D', 'BWD', 'IWD', 'LBD', 'PSD', 'WHD', 'SJD', 'VCD', 'GBD', 'BSD']
away_odds_labels = ['B365A', 'BWA', 'IWA', 'LBA', 'PSA', 'WHA', 'SJA', 'VCA', 'GBA', 'BSA']
all_odds_labels = [home_odds_labels, draw_odds_labels, away_odds_labels]
combined_odds_labels = home_odds_labels + draw_odds_labels + away_odds_labels

matches = recent_matches[combined_odds_labels]

In [360]:
predictions = pd.DataFrame(columns=['home_prob', 'draw_prob', 'away_prob'])

for i in range(len(matches)):
    vals = np.array([np.random.rand() for i in range(3)])
    vals /= np.sum(vals)
    predictions.loc[i, 'home_prob'] = vals[0]
    predictions.loc[i, 'draw_prob'] = vals[1]
    predictions.loc[i, 'away_prob'] = vals[2]

In [361]:
home_, draw_, away_ = (0, 1, 2)

def get_result_corresponding_to_betting_org(org):
    for result, odds_orgs in enumerate(all_odds_labels):
        if org in odds_orgs:
            return result


def get_bet_org(odds, predictions):
    best_exp, best_org, best_odds, chosen_result = 0, None, 0, None
    for org, odds_ in odds.iteritems():
        result = get_result_corresponding_to_betting_org(org)
        exp = odds_ * predictions[result]
        if exp > best_exp:
            best_exp, best_org, best_odds, chosen_result = exp, org, odds_, result
            
    return chosen_result, best_org
    
    
def get_bet(odds, predictions, funds):
    bet_side, bet_org = get_bet_org(odds, predictions)
    bet_amount = np.sqrt(funds) * odds[bet_org] * predictions[bet_side]
    bet_amount = min(funds/2, bet_amount)
    
    return bet_amount, bet_side, bet_org


def get_profit(bet_amount, bet_side, odds, labels_row):
    if labels_row[bet_side]:
        return bet_amount * odds
    else:
        return -bet_amount

In [362]:
funds = 50
initial_funds = funds

n_correct = 0

for r in range(len(matches)):
    bet_amount, bet_side, bet_org = get_bet(matches.loc[r], predictions.loc[r], funds)
    odds =  matches.loc[r, bet_org]
    profit = get_profit(bet_amount, bet_side, odds, labels.loc[r])
    funds += profit
    
    if funds < 1:
        break
    
    if profit > 0:
        n_correct += 1
    
    
profit = funds - initial_funds
print('initial funds: ${:.2f}'.format(initial_funds))
print('final funds: ${:.2f}'.format(funds))
print('profit: ${0:.2f}'.format(profit))
print('profit percent: {0:.2f}%'.format(profit * 100 / initial_funds))
print('predicted {} of {} matches correctly'.format(n_correct, r))
print('prediction percent: {0:.2f}%'.format(n_correct * 100 / r))
print('lasted {} of {} matches before dropping below $1'.format(r, len(matches)))
print('made it through {0:.2f}% of matches before dropping below 1 dollar'.format(100 * r / len(matches)))

initial funds: $50.00
final funds: $0.99
profit: $-49.01
profit percent: -98.03%
predicted 1 of 12 matches correctly
prediction percent: 8.33%
lasted 12 of 520 matches before dropping below $1
made it through 2.31% of matches before dropping below 1 dollar
